## <p style="text-align: center;"> Pathing using OpenCV in Python - Ants example</p>

### Author: Venkatesh Chandra

<div class="alert alert-block alert-info">
<b>Motion Detection:</b> Detect and Track the motion of ants
</div>

#### Description

This code detects objects and tracks their path

#### Import the necessary packages

In [10]:
import cv2
import math
import copy
import numpy as np
from collections import deque
import numpy as np
import argparse
import imutils

#### Check the neighbours and combine them

In [11]:
def check_neighbour(cnt1,cnt2):
    row1,row2 = cnt1.shape[0],cnt2.shape[0]
    for i in range(row1):
        for j in range(row2):
            dist = np.linalg.norm(cnt1[i]-cnt2[j])
            if abs(dist) < 50 :
                return True
            elif i==row1-1 and j==row2-1:
                return False

#### Specify the color range in HSV code

In [12]:
blackLower = np.array([0,0,0])
blackUpper = np.array([180, 255, 30])


points = deque(maxlen=32)
counter = 0

#### Define the output of the video
<br> Pick up any video from royalty free websites that has ants in black color (or specify the color range) in a contrasting background

In [13]:
video_capture = cv2.VideoCapture(r"C:\Users\vchan\OneDrive - McGill University\Image-Processing-master/Ants3_input.mp4")

fourcc = cv2.VideoWriter_fourcc('m','p','4','v')# note the lower case
frame_width = int(video_capture.get(3))
frame_height = int(video_capture.get(4))
out = cv2.VideoWriter('Ants_tracking_output.mp4',fourcc , 10, (frame_width,frame_height), True)

captured_frame = []
captured_frame.append(deque())
frameIndex = 0

#### Main Loop

In [14]:
while True:
    # start reading the camera
    ret, image = video_capture.read()
    if not ret:
        break

    # Change to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    #Create masks with coordinates to detect the color
    mask = cv2.inRange(hsv,blackLower,blackUpper)
    
    counts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)[-2]

    center = None
    
    #Check if we have one contour, and then proceed
    if len(counts) > 0:

        LENGTH = len(counts)
        zeros_array = np.zeros((LENGTH, 1))
        for i, count1 in enumerate(counts):
            x = i
            if i != LENGTH - 1:
                for j, count2 in enumerate(counts[i + 1:]):
                    x = x + 1
                    distance = check_neighbour(count1, count2)
                    if distance == True:
                        val = min(zeros_array[i], zeros_array[x])
                        zeros_array[x] = zeros_array[i] = val
                    else:
                        if zeros_array[x] == zeros_array[i]:
                            zeros_array[x] = i + 1

        unified = []
        maximum = int(zeros_array.max()) + 1
        for i in range(maximum):
            pos = np.where(zeros_array == i)[0]
            if pos.size != 0:
                c = np.vstack(counts[i] for i in pos)
                hull = cv2.convexHull(c)
                unified.append(hull)

            ((x, y), radius) = cv2.minEnclosingCircle(c)
            M = cv2.moments(c)
            circle_appears= "no"
            if(len(counts) >2):
                if(M["m00"] >0):
                    center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                    circle_appears = "yes"
                else:
                    center = (counts[0][:, 0][0, 0], counts[0][:, 0][0, 1])
            else:
                center =(counts[0][:,0][0,0],counts[0][:,0][0,1])

            if(circle_appears is "yes"):
                cricle1 = cv2.circle(image, (int(x), int(y)), int(50),
                           (0, 255, 255), 2)

                points.appendleft((int(x), int(y)))
                if captured_frame is not None:
                    captured_frame[frameIndex].appendleft(copy.deepcopy((int(x), int(y))))

    if frameIndex > 0:
        for j in range(0, frameIndex):
            if len(captured_frame[j]) < len(captured_frame[j - 1]):
                min1 = len(captured_frame[j])
            else:
                min1 = len(captured_frame[j - 1])
            for i in np.arange(0, min1):

                if j >0 and len(captured_frame[j])>0 :
                    distance = math.hypot(captured_frame[j][i][0] - captured_frame[j-1][i][0], captured_frame[j][i][1] - captured_frame[j-1][i][1])
                    if distance <35:
                        cv2.line(image, captured_frame[j-1][i], captured_frame[j][i], (0, 0, 255), 5)

    if len(captured_frame[frameIndex]) > 0:
        captured_frame.append(deque())
        frameIndex += 1

    cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('Frame', 500, 500)
    cv2.imshow("Frame", image)
    name = r"C:\Users\vchan\OneDrive - McGill University\Image-Processing-master/Frames/frameoutput%d.jpg" % counter
    cv2.imwrite(name, image)
    
    counter += 1

    if cv2.waitKey(25) == 13:
        break
        
#Release the video handle
video_capture.release()
cv2.destroyAllWindows()

C:\Users\vchan\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


#### Now you have the frames saved as images in the folder specified. Next, we run a loop to convert the frames to a video

In [15]:
for m in range(0,236):
    out.write(cv2.imread(r"C:\Users\vchan\OneDrive - McGill University\Image-Processing-master/Frames/frameoutput%d.jpg" % m))

#### Release all handles

In [16]:
out.release()
cv2.destroyAllWindows()

## Notes for adding colors

###### Red color
low_red = np.array([161, 155, 84])
high_red = np.array([179, 255, 255])
red_mask = cv2.inRange(hsv_frame, low_red, high_red)
red = cv2.bitwise_and(frame, frame, mask=red_mask)

###### Blue color
low_blue = np.array([94, 80, 2])
high_blue = np.array([126, 255, 255])
blue_mask = cv2.inRange(hsv_frame, low_blue, high_blue)
blue = cv2.bitwise_and(frame, frame, mask=blue_mask)

###### Green color
low_green = np.array([25, 52, 72])
high_green = np.array([102, 255, 255])
green_mask = cv2.inRange(hsv_frame, low_green, high_green)
green = cv2.bitwise_and(frame, frame, mask=green_mask)

###### Every color except white
low = np.array([0, 42, 0])
high = np.array([179, 255, 255])
mask = cv2.inRange(hsv_frame, low, high)
result = cv2.bitwise_and(frame, frame, mask=mask)